###Desenvolvimento e Avaliação de uma Arquitetura Distribuída para o Cadastro Ambiental Rural

In [0]:
# Criação da pasta "CADASTRO AMBIENTAL" em /FileStore/tables/
dbutils.fs.mkdirs("/FileStore/tables/cadastro_ambiental")

True

In [0]:
# Upload do arquivo na pasta do projeto

In [0]:
display(dbutils.fs.ls("/FileStore/tables/cadastro_ambiental"))

path,name,size,modificationTime
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/,delta_temas_ambientais/,0,0
dbfs:/FileStore/tables/cadastro_ambiental/gold_temas_ambientais/,gold_temas_ambientais/,0,0
dbfs:/FileStore/tables/cadastro_ambiental/temas_ambientais_csv.gz,temas_ambientais_csv.gz,731449146,1733014308000


In [0]:
# Definindo os Caminhos
compressed_file_path = "/FileStore/tables/cadastro_ambiental/temas_ambientais_csv.gz"
delta_table_path = "/FileStore/tables/cadastro_ambiental/delta_temas_ambientais"


In [0]:
display(dbutils.fs.ls("/FileStore/tables/cadastro_ambiental/temas_ambientais_csv.gz"))

path,name,size,modificationTime
dbfs:/FileStore/tables/cadastro_ambiental/temas_ambientais_csv.gz,temas_ambientais_csv.gz,731449146,1733014308000


In [0]:
# Leitura do Arquivo GZIP / CSV
from pyspark.sql import SparkSession

# Criar sessão Spark
spark = SparkSession.builder.appName("ProcessarCadastroAmbiental").getOrCreate()

# Ler o arquivo CSV compactado
# Define que o CSV tem cabeçalhos
# Inferir automaticamente o tipo de dados
# Define o delimitador como ";"
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ";") \
    .load(compressed_file_path)

# Mostrar os dados para validar
df.limit(5).display()


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
GO,Nazário,5214408,119.6326,GO-5214408-3AEF2043582E40238C0F84A553686CA7,AT,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",119.6326,6.43202795367569,6.432,5.29751084282892,0.00102295610380904,Sim,-16.5923058689987,-49.9019017039191,2014-05-07T16:01:44.305Z,null,112.301149046683,0.0,IRU,5.4378,0.0,0.0,0.0,0.0,2014-05-07T16:01:44.305Z
SC,Meleiro,4210803,7.534,SC-4210803-BC127B0EC8DB49AC9D46D723286241A2,AT,"Aguardando análise, não passível de revisão de dados",7.534,5.51785410336982,0.0,0.0,0.00159349317984888,Nao,-28.7930798512303,-49.6472023744097,2014-05-07T16:02:02.915Z,null,2.01445162492469,0.0,IRU,0.4186,0.0,1.506,0.0,0.0,2014-05-07T16:02:02.915Z
GO,Nova Roma,5214903,19.4883,GO-5214903-7F58049BD79046E9A904CC81C5AC177A,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",19.36,19.4882633569717,3.872,0.0,0.0,Nao,-13.6370551503248,-47.0339670619739,2014-05-07T16:06:15.777Z,null,0.0,0.0,IRU,0.2784,0.0,0.0,0.0,0.0,2014-05-07T16:06:15.777Z
GO,Santa Helena de Goiás,5219308,22.934,GO-5219308-6478196E75CF4F65800ACA0758575820,PE,"Analisado com pendências, aguardando retificação",22.88,1.5021496193707,1.5028,0.568927694143718,18.2335997514784,Sim,-17.8327877579625,-50.6015145422339,2014-05-07T17:49:36.938Z,null,3.08289042473435,0.0,IRU,1.1467,0.0,0.0,0.0,0.0,2014-05-07T17:49:36.938Z
PR,Cornélio Procópio,4106407,10.956,PR-4106407-0F06081500254BE3A479EE8EFFDD5319,AT,Em análise,10.956,0.0,0.0,0.0,0.161657419734154,Nao,-23.1841101335722,-50.6715645967419,2014-05-07T17:52:55.333Z,null,10.7942669028953,0.0,IRU,0.6087,0.0,0.0,0.0,0.0,2014-05-07T17:52:55.333Z


In [0]:
df.printSchema()

root
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- codigo_ibge: integer (nullable = true)
 |-- area_do_imovel: double (nullable = true)
 |-- registro_car: string (nullable = true)
 |-- situacao_cadastro: string (nullable = true)
 |-- condicao_cadastro: string (nullable = true)
 |-- area_liquida: double (nullable = true)
 |-- area_remanescente_vegetacao_nativa: double (nullable = true)
 |-- area_reserva_legal_proposta: double (nullable = true)
 |-- area_preservacao_permanente: double (nullable = true)
 |-- area_nao_classificada: double (nullable = true)
 |-- solicitacao_adesao_pra: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- data_inscricao: timestamp (nullable = true)
 |-- data_alteracao_condicao_cadastro: timestamp (nullable = true)
 |-- area_rural_consolidada: double (nullable = true)
 |-- area_servidao_administrativa: double (nullable = true)
 |-- tipo_imovel_rural: string (nullable =

In [0]:
df.dtypes

[('uf', 'string'),
 ('municipio', 'string'),
 ('codigo_ibge', 'int'),
 ('area_do_imovel', 'double'),
 ('registro_car', 'string'),
 ('situacao_cadastro', 'string'),
 ('condicao_cadastro', 'string'),
 ('area_liquida', 'double'),
 ('area_remanescente_vegetacao_nativa', 'double'),
 ('area_reserva_legal_proposta', 'double'),
 ('area_preservacao_permanente', 'double'),
 ('area_nao_classificada', 'double'),
 ('solicitacao_adesao_pra', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('data_inscricao', 'timestamp'),
 ('data_alteracao_condicao_cadastro', 'timestamp'),
 ('area_rural_consolidada', 'double'),
 ('area_servidao_administrativa', 'double'),
 ('tipo_imovel_rural', 'string'),
 ('modulos_fiscais', 'double'),
 ('area_uso_restrito', 'double'),
 ('area_reserva_legal_averbada', 'double'),
 ('area_reserva_legal_aprovada_nao_averbada', 'double'),
 ('area_pousio', 'double'),
 ('data_ultima_retificacao', 'timestamp')]

In [0]:
df.count()

6839104

In [0]:
# Salvar os dados como Delta Lake
df.write.format("delta").mode("overwrite").save(delta_table_path)

In [0]:
display(dbutils.fs.ls("/FileStore/tables/cadastro_ambiental/delta_temas_ambientais"))

path,name,size,modificationTime
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-385bc86e-1629-430e-b76f-1bf11b97c69d-c000.snappy.parquet,part-00000-385bc86e-1629-430e-b76f-1bf11b97c69d-c000.snappy.parquet,808912071,1733103685000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-464a0156-62ca-47e3-ba86-f58ddfc326a7-c000.snappy.parquet,part-00000-464a0156-62ca-47e3-ba86-f58ddfc326a7-c000.snappy.parquet,808912071,1733158736000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-565de7a1-4107-41ca-891b-ce333bc650c1-c000.snappy.parquet,part-00000-565de7a1-4107-41ca-891b-ce333bc650c1-c000.snappy.parquet,808912071,1733149927000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-74a039b1-e08c-46d3-be8d-419131535315-c000.snappy.parquet,part-00000-74a039b1-e08c-46d3-be8d-419131535315-c000.snappy.parquet,808912071,1733104312000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-a9898c3f-aa2a-48d6-a837-c1213e24333f-c000.snappy.parquet,part-00000-a9898c3f-aa2a-48d6-a837-c1213e24333f-c000.snappy.parquet,808912071,1733068596000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-bd5ff4a3-2fe1-4bf8-b5f8-63402f6ccbe3-c000.snappy.parquet,part-00000-bd5ff4a3-2fe1-4bf8-b5f8-63402f6ccbe3-c000.snappy.parquet,808912071,1733017443000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-d3558dc1-dd88-4ff4-b709-2146c5975cbc-c000.snappy.parquet,part-00000-d3558dc1-dd88-4ff4-b709-2146c5975cbc-c000.snappy.parquet,808912071,1733052775000
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/part-00000-fd45aec3-7982-4f68-973b-217b3ef6721e-c000.snappy.parquet,part-00000-fd45aec3-7982-4f68-973b-217b3ef6721e-c000.snappy.parquet,808912071,1733062446000


###Arquitetura modelo Lakehouse com a estrutura Bronze-Silver-Gold para organização dos dados

In [0]:
# Caminho da camada Bronze (dados brutos)
bronze_path = "/FileStore/tables/cadastro_ambiental/delta_temas_ambientais"

# Caminho da camada Silver (dados transformados)
silver_path = "/FileStore/tables/cadastro_ambiental/silver_temas_ambientais"


In [0]:
from pyspark.sql import functions as F

# Carregar os dados da camada Bronze
df_bronze = spark.read.format("delta").load(bronze_path)

In [0]:
from pyspark.sql import functions as F

# Contar linhas com latitude ou longitude nulas
null_latitude_count = df_bronze.filter(F.col("latitude").isNull()).count()
null_longitude_count = df_bronze.filter(F.col("longitude").isNull()).count()

# Exibir os resultados
print(f"Total de linhas com latitude nula: {null_latitude_count}")
print(f"Total de linhas com longitude nula: {null_longitude_count}")


Total de linhas com latitude nula: 0
Total de linhas com longitude nula: 0


In [0]:
# Listar arquivos no caminho para verificar se foi removido
display(dbutils.fs.ls("/FileStore/tables/cadastro_ambiental/"))


path,name,size,modificationTime
dbfs:/FileStore/tables/cadastro_ambiental/delta_temas_ambientais/,delta_temas_ambientais/,0,0
dbfs:/FileStore/tables/cadastro_ambiental/gold_temas_ambientais/,gold_temas_ambientais/,0,0
dbfs:/FileStore/tables/cadastro_ambiental/temas_ambientais_csv.gz,temas_ambientais_csv.gz,731449146,1733014308000


In [0]:
# Contar registros únicos por UF
unique_uf_count = df_bronze.select("uf").distinct().count()
print(f"Total de registros únicos por UF: {unique_uf_count}")

Total de registros únicos por UF: 27


In [0]:
# Contar registros únicos por Municipio
unique_municipio_count = df_bronze.select("municipio").distinct().count()
print(f"Total de registros únicos por Município: {unique_municipio_count}")

Total de registros únicos por Município: 5287


In [0]:
# Contagem individual de registros por UF
df_count_by_uf = df_bronze.groupBy("uf").count()

# Contagem individual de registros por Municipio
df_count_by_municipio = df_bronze.groupBy("municipio").count()

# Exibir resultados
print("Contagem por UF:")
df_count_by_uf.orderBy("count", ascending=False).show(truncate=False)

print("Contagem por Município:")
df_count_by_municipio.orderBy("count", ascending=False).show(truncate=False)

Contagem por UF:
+---+-------+
|uf |count  |
+---+-------+
|BA |1012499|
|MG |989446 |
|RS |606454 |
|PR |496164 |
|SP |408528 |
|SC |374918 |
|PE |339371 |
|CE |314866 |
|PA |279598 |
|MA |273642 |
|PI |255052 |
|GO |199690 |
|PB |170851 |
|MT |170205 |
|RO |147287 |
|AL |114610 |
|ES |108849 |
|SE |94418  |
|RN |89357  |
|TO |85537  |
+---+-------+
only showing top 20 rows

Contagem por Município:
+------------------+-----+
|municipio         |count|
+------------------+-----+
|Monte Santo       |19220|
|Brasília          |17339|
|Porto Velho       |16906|
|Canguçu           |13503|
|Araripina         |12183|
|Pilão Arcado      |11928|
|Euclides da Cunha |11361|
|Ouricuri          |11324|
|Brumado           |11019|
|Montes Claros     |10724|
|Tucano            |10256|
|Teresina          |10025|
|Jeremoabo         |9974 |
|Rio Pardo de Minas|9619 |
|Araci             |9489 |
|Prudentópolis     |9470 |
|Campo Formoso     |9445 |
|Macaúbas          |9350 |
|São Félix do Xingu|9321 |
|Ja

In [0]:
# Caminho da camada Gold
gold_path = "/FileStore/tables/cadastro_ambiental/gold_temas_ambientais"

# Salvar os dados na Camada Gold particionados por UF
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("uf") \
    .save(gold_path)

In [0]:
# Carregar os dados particionados da Camada Gold
df_gold = spark.read.format("delta").load(gold_path)

# Exibir os dados carregados
df_gold.limit(5).display()

uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
AC,Xapuri,1200708,935.0576,AC-1200708-19C3C6A0A7B6488096185809637AC4AF,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",934.4761,0.0,0.0,61.488719378311,6.52856406392798,Sim,-10.7142027389389,-68.2084015961516,2014-05-30T21:05:38.957Z,2022-01-06T19:50:28.955Z,891.99866751478,0.0,IRU,9.3506,0.0,0.0,0.0,0.0,2014-05-30T21:05:38.957Z
AC,Capixaba,1200179,295.308,AC-1200179-CF396FF9F78E4A0E8CCA9CF80716144A,AT,"Analisado com pendências, aguardando apresentação de documentos",295.1906,23.6549331578795,0.0,0.0,0.205561556893404,Sim,-10.5652398887943,-67.8238254166808,2014-05-30T21:05:39.074Z,null,271.447441579244,0.0,IRU,4.2187,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.074Z
AC,Bujari,1200138,2.9585,AC-1200138-218DCFB19D064EF38E435A4281FCEBB4,AT,"Aguardando análise, não passível de revisão de dados",2.9585,0.92996846870482,0.0,0.0,0.0549187275759037,Sim,-9.74302633370553,-68.0912357095767,2014-05-30T21:05:39.141Z,null,1.97361999899,0.0,IRU,0.0423,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.141Z
AC,Sena Madureira,1200500,4528.6282,AC-1200500-FB62BF39455A442591A7D60DBCFA800C,AT,"Analisado com pendências, aguardando apresentação de documentos",4525.0442,4315.24535000182,3621.8188,85.2533067661144,0.0164362580727577,Sim,-9.85635101020226,-68.8012355599982,2014-05-30T21:05:39.368Z,null,198.873999505585,0.0,IRU,45.2863,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.368Z
AC,Rio Branco,1200401,8298.6541,AC-1200401-69A98AA176E24BC8990CEFB9417A8A4A,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",8244.4361,5324.53362373621,5306.5103,99.9105088240575,0.0281518691906137,Sim,-9.87951925357666,-68.6313721346212,2014-05-30T21:05:39.681Z,null,2909.13662900035,47.4914,IRU,118.5522,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.681Z


In [0]:
# Realizar a agregação por UF na camada Gold
df_gold_aggregated = df_gold.groupBy("uf") \
    .agg(
        F.sum("area_do_imovel").alias("total_area_imovel"),
        F.sum("area_liquida").alias("total_area_liquida"),
        F.count("*").alias("total_registros")
    )

In [0]:
df_gold_aggregated.display()

uf,total_area_imovel,total_area_liquida,total_registros
MG,5.4337736262100935E7,5.296282176140109E7,989446
BA,3.576177102590188E7,3.489551777910152E7,1012499
RS,2.3677584359099682E7,2.341407737939912E7,606454
PR,1.8899704235899527E7,1.8640516105399292E7,496164
SC,8254598.620099617,8136270.711699749,374918
SP,2.3335330627499703E7,2.221957907379932E7,408528
PE,7334876.880099971,7205341.76689996,339371
CE,1.1065764441099936E7,1.0755713251900053E7,314866
PI,2.0116996145900246E7,1.9793940273700487E7,255052
PA,8.213900892529853E7,7.454478326700152E7,279598


In [0]:
# Validar os dados da Camada Gold
df_gold = spark.read.format("delta").load(gold_path)
df_gold.limit(5).display()

uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
AC,Xapuri,1200708,935.0576,AC-1200708-19C3C6A0A7B6488096185809637AC4AF,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",934.4761,0.0,0.0,61.488719378311,6.52856406392798,Sim,-10.7142027389389,-68.2084015961516,2014-05-30T21:05:38.957Z,2022-01-06T19:50:28.955Z,891.99866751478,0.0,IRU,9.3506,0.0,0.0,0.0,0.0,2014-05-30T21:05:38.957Z
AC,Capixaba,1200179,295.308,AC-1200179-CF396FF9F78E4A0E8CCA9CF80716144A,AT,"Analisado com pendências, aguardando apresentação de documentos",295.1906,23.6549331578795,0.0,0.0,0.205561556893404,Sim,-10.5652398887943,-67.8238254166808,2014-05-30T21:05:39.074Z,null,271.447441579244,0.0,IRU,4.2187,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.074Z
AC,Bujari,1200138,2.9585,AC-1200138-218DCFB19D064EF38E435A4281FCEBB4,AT,"Aguardando análise, não passível de revisão de dados",2.9585,0.92996846870482,0.0,0.0,0.0549187275759037,Sim,-9.74302633370553,-68.0912357095767,2014-05-30T21:05:39.141Z,null,1.97361999899,0.0,IRU,0.0423,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.141Z
AC,Sena Madureira,1200500,4528.6282,AC-1200500-FB62BF39455A442591A7D60DBCFA800C,AT,"Analisado com pendências, aguardando apresentação de documentos",4525.0442,4315.24535000182,3621.8188,85.2533067661144,0.0164362580727577,Sim,-9.85635101020226,-68.8012355599982,2014-05-30T21:05:39.368Z,null,198.873999505585,0.0,IRU,45.2863,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.368Z
AC,Rio Branco,1200401,8298.6541,AC-1200401-69A98AA176E24BC8990CEFB9417A8A4A,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",8244.4361,5324.53362373621,5306.5103,99.9105088240575,0.0281518691906137,Sim,-9.87951925357666,-68.6313721346212,2014-05-30T21:05:39.681Z,null,2909.13662900035,47.4914,IRU,118.5522,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.681Z


In [0]:
spark.sql(f"""
    CREATE TABLE gold_temas_ambientais
    USING DELTA
    LOCATION '{gold_path}'
""")

DataFrame[]

In [0]:
# Carregar os dados agregados da Camada Gold
df_gold = spark.read.format("delta").load(gold_path)

# Exibir os dados agregados
df_gold.limit(5).display()

# Consultar os dados via SQL
spark.sql("SELECT * FROM gold_temas_ambientais LIMIT 10").show()


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
AC,Xapuri,1200708,935.0576,AC-1200708-19C3C6A0A7B6488096185809637AC4AF,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",934.4761,0.0,0.0,61.488719378311,6.52856406392798,Sim,-10.7142027389389,-68.2084015961516,2014-05-30T21:05:38.957Z,2022-01-06T19:50:28.955Z,891.99866751478,0.0,IRU,9.3506,0.0,0.0,0.0,0.0,2014-05-30T21:05:38.957Z
AC,Capixaba,1200179,295.308,AC-1200179-CF396FF9F78E4A0E8CCA9CF80716144A,AT,"Analisado com pendências, aguardando apresentação de documentos",295.1906,23.6549331578795,0.0,0.0,0.205561556893404,Sim,-10.5652398887943,-67.8238254166808,2014-05-30T21:05:39.074Z,null,271.447441579244,0.0,IRU,4.2187,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.074Z
AC,Bujari,1200138,2.9585,AC-1200138-218DCFB19D064EF38E435A4281FCEBB4,AT,"Aguardando análise, não passível de revisão de dados",2.9585,0.92996846870482,0.0,0.0,0.0549187275759037,Sim,-9.74302633370553,-68.0912357095767,2014-05-30T21:05:39.141Z,null,1.97361999899,0.0,IRU,0.0423,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.141Z
AC,Sena Madureira,1200500,4528.6282,AC-1200500-FB62BF39455A442591A7D60DBCFA800C,AT,"Analisado com pendências, aguardando apresentação de documentos",4525.0442,4315.24535000182,3621.8188,85.2533067661144,0.0164362580727577,Sim,-9.85635101020226,-68.8012355599982,2014-05-30T21:05:39.368Z,null,198.873999505585,0.0,IRU,45.2863,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.368Z
AC,Rio Branco,1200401,8298.6541,AC-1200401-69A98AA176E24BC8990CEFB9417A8A4A,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",8244.4361,5324.53362373621,5306.5103,99.9105088240575,0.0281518691906137,Sim,-9.87951925357666,-68.6313721346212,2014-05-30T21:05:39.681Z,null,2909.13662900035,47.4914,IRU,118.5522,0.0,0.0,0.0,0.0,2014-05-30T21:05:39.681Z


+---+--------------+-----------+--------------+--------------------+-----------------+--------------------+------------+----------------------------------+---------------------------+---------------------------+---------------------+----------------------+-----------------+-----------------+--------------------+--------------------------------+----------------------+----------------------------+-----------------+---------------+-----------------+---------------------------+----------------------------------------+-----------+-----------------------+
| uf|     municipio|codigo_ibge|area_do_imovel|        registro_car|situacao_cadastro|   condicao_cadastro|area_liquida|area_remanescente_vegetacao_nativa|area_reserva_legal_proposta|area_preservacao_permanente|area_nao_classificada|solicitacao_adesao_pra|         latitude|        longitude|      data_inscricao|data_alteracao_condicao_cadastro|area_rural_consolidada|area_servidao_administrativa|tipo_imovel_rural|modulos_fiscais|area_uso_res

###Bateria de Testes (Queries)

In [0]:
# Consulta 1
# Soma de área para MS e MT, em ordem decrescente
query1 = spark.sql("""
    SELECT uf, SUM(area_do_imovel) AS total_area_hectares
    FROM gold_temas_ambientais
    WHERE uf IN ('MS', 'MT')
    GROUP BY uf
    ORDER BY total_area_hectares DESC
""")

# Exibir os resultados
display(query1)


uf,total_area_hectares
MT,8.458708926979955E7
MS,3.632188266120003E7


In [0]:
# Consulta 2
# Filtrar propriedades da região Sudeste
query2 = spark.sql("""
    SELECT *
    FROM gold_temas_ambientais
    WHERE uf IN ('SP', 'RJ', 'MG', 'ES')
""")

# Exibir as primeiras propriedades da região Sudeste
display(query2.limit(10))


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
ES,Ibiraçu,3202504,314.1342,ES-3202504-69D8657F3B1D4CA4B535C417C2308A7E,AT,"Aguardando análise, não passível de revisão de dados",314.1342,0.0,0.0,0.0,314.072377485086,Nao,-19.8525944204436,-40.4928923145537,2016-05-04T22:17:47.635Z,null,0.0,0.0,PCT,4.0,0.0,0.0,0.0,0.0,2016-05-04T22:17:47.635Z
ES,Rio Bananal,3204351,7.99,ES-3204351-0DE30AF546164820B819802C4D55E93A,AT,"Aguardando análise, não passível de revisão de dados",7.9884,2.64,2.64,0.01,0.0,Sim,-19.2473157954533,-40.2852193704247,2017-12-14T00:00:00Z,null,5.33,0.0,IRU,0.4,0.0,0.0,0.0,0.0,2017-12-14T16:33:12.28Z
ES,Domingos Martins,3201902,4.36,ES-3201902-F6BA12CC0C1548F1BAB0A76CB8C18BDE,AT,"Aguardando análise, não passível de revisão de dados",4.3619,1.05,1.05,0.59,7.88638113591905E-4,Sim,-20.3817501007776,-40.8517174413158,2017-12-14T00:00:00Z,null,3.3,0.0,IRU,0.24,0.0,0.0,0.0,0.0,2017-12-14T16:34:18.82Z
ES,São Gabriel da Palha,3204708,83.02,ES-3204708-BBACBBF4B5C348DAAFE457760E87E4B6,AT,"Aguardando análise, não passível de revisão de dados",83.0211,21.89,11.6,0.0,0.0103664973386715,Sim,-19.0181309820505,-40.4552425564567,2017-12-14T00:00:00Z,null,61.17,0.0,IRU,4.15,0.0,5.07,0.0,0.0,2017-12-14T16:34:41.371Z
ES,Ecoporanga,3202108,7.32,ES-3202108-083F1AD7BCFE4D0081A37DCD2027BAA3,AT,"Aguardando análise, não passível de revisão de dados",7.3165,0.0,0.0,0.95,2.60410745378732E-4,Sim,-18.0430755961406,-40.6805885631316,2017-12-14T00:00:00Z,null,7.26,0.0,IRU,0.15,0.0,0.0,0.0,0.0,2017-12-14T16:34:53.355Z
ES,Montanha,3203502,20.47,ES-3203502-6479AB276B614A398BFAC18B6BD613EF,AT,"Aguardando análise, não passível de revisão de dados",20.4742,0.0,0.0,3.13,3.83288103808598E-4,Sim,-18.0305222741377,-40.2157485296771,2017-12-14T00:00:00Z,null,20.27,0.0,IRU,0.34,0.0,0.0,0.0,0.0,2017-12-14T16:34:47.551Z
ES,Ponto Belo,3204252,4.93,ES-3204252-91D468427F314CC6A9130AA231017139,AT,"Aguardando análise, não passível de revisão de dados",4.9335,0.0,0.0,1.35,0.0,Sim,-18.2481057455036,-40.5323885599816,2017-12-14T00:00:00Z,null,4.89,0.0,IRU,0.08,0.0,0.0,0.0,0.0,2017-12-14T16:35:29.54Z
ES,Colatina,3201506,69.72,ES-3201506-7CD51BAFE1A94152ABDE85080AE210AA,AT,"Aguardando análise, não passível de revisão de dados",69.7161,14.69,10.81,6.4,0.0054509336059735,Sim,-19.4271525790449,-40.8261142827437,2017-12-14T00:00:00Z,null,54.73,0.0,IRU,3.87,0.0,0.0,0.0,0.0,2017-12-14T16:35:37.342Z
ES,São Gabriel da Palha,3204708,9.27,ES-3204708-92FD2FBCF66E4365AF987A5EB281F74C,AT,"Aguardando análise, não passível de revisão de dados",9.2741,3.47,2.05,0.0,1.02395404866908,Sim,-18.9339233345634,-40.397457495568,2016-05-02T00:00:00Z,null,4.77,0.0,IRU,0.46,0.0,0.0,0.0,0.0,2017-12-14T19:11:30.332Z
ES,Anchieta,3200409,17.85,ES-3200409-BB756500D56243EE8C2CE94467E6112C,AT,"Aguardando análise, não passível de revisão de dados",17.8481,0.81,0.81,4.23,0.0,Sim,-20.7902874422207,-40.675579786973,2016-05-02T00:00:00Z,null,17.04,0.0,IRU,1.12,0.0,0.0,0.0,0.0,2017-12-14T19:11:38.152Z


In [0]:
# Consulta 3

from pyspark.sql.functions import col

# Definir coordenadas do polígono (usaremos bounds para simplificar)
min_lon, max_lon = -53.8181518, -51.0495971
min_lat, max_lat = -19.4632582, -16.1924262

# Filtrar propriedades dentro do bounding box do polígono
query3 = spark.sql(f"""
    SELECT *
    FROM gold_temas_ambientais
    WHERE longitude BETWEEN {min_lon} AND {max_lon}
      AND latitude BETWEEN {min_lat} AND {max_lat}
""")

# Exibir os resultados
display(query3.limit(10))


uf,municipio,codigo_ibge,area_do_imovel,registro_car,situacao_cadastro,condicao_cadastro,area_liquida,area_remanescente_vegetacao_nativa,area_reserva_legal_proposta,area_preservacao_permanente,area_nao_classificada,solicitacao_adesao_pra,latitude,longitude,data_inscricao,data_alteracao_condicao_cadastro,area_rural_consolidada,area_servidao_administrativa,tipo_imovel_rural,modulos_fiscais,area_uso_restrito,area_reserva_legal_averbada,area_reserva_legal_aprovada_nao_averbada,area_pousio,data_ultima_retificacao
GO,Caiapônia,5204409,195.8985,GO-5204409-6924D519EF9244648EB009539D3D9753,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",195.6028,193.040465236806,38.4985,16.3696915324486,0.0,Nao,-17.118992154096,-51.5959819982595,2014-05-11T09:27:48.5Z,null,0.0,0.0,IRU,3.265,0.0,0.0,0.0,0.0,2014-05-11T09:27:48.5Z
GO,Caiapônia,5204409,391.9673,GO-5204409-29C77215AF204099B40658289E1F09BC,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",391.3813,389.260278100578,77.5853,16.0453100108765,0.0,Nao,-17.1232355286134,-51.6111926571558,2014-05-11T10:59:26.731Z,null,0.0,0.0,IRU,6.5328,0.0,0.0,0.0,0.0,2014-05-11T10:59:26.731Z
GO,Caiapônia,5204409,108.1409,GO-5204409-05CDDE0D903540289792DED4B60D7FC6,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",108.0315,107.928773218584,21.4538,1.35032117744833,0.0,Nao,-17.2272688678909,-52.0611379478198,2014-05-11T11:10:43.313Z,null,0.0,0.0,IRU,1.8023,0.0,0.0,0.0,0.0,2014-05-11T11:10:43.313Z
GO,Caiapônia,5204409,422.0878,GO-5204409-48DE25DEE5B8490D93CF90F79022266E,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",421.4525,422.087792600632,84.3033,0.0,0.0,Nao,-17.0923642253856,-51.5989472024192,2014-05-11T11:12:47.628Z,null,0.0,0.0,IRU,7.0348,0.0,0.0,0.0,0.0,2014-05-11T11:12:47.628Z
GO,Caiapônia,5204409,545.2243,GO-5204409-5563DDF5891E4CE4A39A83EA65E07904,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",544.3281,544.491653470397,108.2674,4.51203189095352,0.0,Nao,-17.11702846203,-51.4593113547756,2014-05-11T11:15:01.836Z,null,0.0,0.0,IRU,9.0871,0.0,0.0,0.0,0.0,2014-05-11T11:15:01.836Z
GO,Caiapônia,5204409,197.5993,GO-5204409-79418BE625514399A28DC302DC17F013,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",197.2709,197.599297358608,39.5022,83.6170568057138,0.0,Nao,-17.1671264858525,-51.4378006049413,2014-05-11T11:16:43.969Z,null,0.0,0.0,IRU,3.2933,0.0,0.0,0.0,0.0,2014-05-11T11:16:43.969Z
GO,Caiapônia,5204409,97.3442,GO-5204409-8AC8D2CCE04744C49318AE9480D058E4,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",97.1972,96.37477998854,19.5018,5.79680014259443,0.0,Nao,-17.1298544449775,-51.5940427189541,2014-05-11T11:18:22.891Z,null,0.0,0.0,IRU,1.6224,0.0,0.0,0.0,0.0,2014-05-11T11:18:22.891Z
GO,Caiapônia,5204409,143.0842,GO-5204409-13F0DDE3F2054674AB625FF960F7B8F9,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",142.8692,143.084231577808,28.5732,0.0,0.0,Nao,-17.1136275529193,-51.6019897003818,2014-05-11T11:22:30.733Z,null,0.0,0.0,IRU,2.3847,0.0,0.0,0.0,0.0,2014-05-11T11:22:30.733Z
GO,Caiapônia,5204409,560.3515,GO-5204409-9CE8FB5D96F34BA3ACD4194FD1323593,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",559.4653,560.351523708534,111.9016,0.0,0.0,Nao,-17.1151440909676,-51.5220864541221,2014-05-11T11:39:58.246Z,null,0.0,0.0,IRU,9.3392,0.0,0.0,0.0,0.0,2014-05-11T11:39:58.246Z
GO,Caiapônia,5204409,228.4778,GO-5204409-593FCAE404DC4C2ABAA69E48C081B4E0,PE,"Analisado com pendências, aguardando retificação e/ou apresentação de documentos",228.102,228.477839460278,45.6202,0.0,0.0,Nao,-17.1323942237996,-51.4568250070597,2014-05-11T12:12:00.17Z,null,0.0,0.0,IRU,3.808,0.0,0.0,0.0,0.0,2014-05-11T12:12:00.17Z


In [0]:
# Consulta 4
# Contagem de propriedades cadastradas por ano
query4 = spark.sql("""
    SELECT YEAR(data_inscricao) AS ano_cadastro, COUNT(*) AS total_propriedades
    FROM gold_temas_ambientais
    GROUP BY ano_cadastro
    ORDER BY ano_cadastro
""")

# Exibir os resultados
display(query4)


ano_cadastro,total_propriedades
null,12
2013,743
2014,227468
2015,1240178
2016,1828786
2017,794779
2018,724856
2019,746570
2020,463322
2021,443942


In [0]:
# Consulta 5
# Percentual médio de área remanescente de vegetação nativa
query5 = spark.sql("""
    SELECT AVG(area_remanescente_vegetacao_nativa / area_do_imovel * 100) AS percentual_medio_vegetacao
    FROM gold_temas_ambientais
""")

# Exibir os resultados
display(query5)


percentual_medio_vegetacao
18.087405591761677


In [0]:
# Consulta 6
# Contagem de propriedades por estado
query6 = spark.sql("""
    SELECT uf, COUNT(*) AS total_propriedades
    FROM gold_temas_ambientais
    GROUP BY uf
    ORDER BY total_propriedades DESC
""")

# Exibir os resultados
query6.show()


+---+------------------+
| uf|total_propriedades|
+---+------------------+
| BA|           1012499|
| MG|            989446|
| RS|            606454|
| PR|            496164|
| SP|            408528|
| SC|            374918|
| PE|            339371|
| CE|            314866|
| PA|            279598|
| MA|            273642|
| PI|            255052|
| GO|            199690|
| PB|            170851|
| MT|            170205|
| RO|            147287|
| AL|            114610|
| ES|            108849|
| SE|             94418|
| RN|             89357|
| TO|             85537|
+---+------------------+
only showing top 20 rows



In [0]:
# Consulta 7
import math

# Define a função para calcular a distância
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Identificar a maior propriedade
query7_max = spark.sql("""
    SELECT latitude, longitude, area_do_imovel
    FROM gold_temas_ambientais
    ORDER BY area_do_imovel DESC
    LIMIT 1
""").collect()

# Coordenadas da maior propriedade
largest_property = query7_max[0]
lat_prop, lon_prop, area = largest_property["latitude"], largest_property["longitude"], largest_property["area_do_imovel"]

# Calcular a distância até Brasília
distance_to_brasilia = haversine(lat_prop, lon_prop, -15.796943053171708, -47.891638482569476)

# Exibir os resultados
print(f"A maior propriedade tem {area} hectares e está localizada a {distance_to_brasilia:.2f} km de Brasília.")


A maior propriedade tem 2420078.7379 hectares e está localizada a 2562.13 km de Brasília.


In [0]:
# Consulta 8
# Calcular a média de área total
avg_area = spark.sql("""
    SELECT AVG(area_do_imovel) AS media_area_total
    FROM gold_temas_ambientais
""").collect()[0]["media_area_total"]

# Filtrar propriedades acima da média por estado
query8 = spark.sql(f"""
    SELECT uf, COUNT(*) AS total_propriedades
    FROM gold_temas_ambientais
    WHERE area_do_imovel > {avg_area}
    GROUP BY uf
    ORDER BY total_propriedades DESC
""")

# Exibir os resultados
display(query8)

uf,total_propriedades
MG,107875
PA,86698
MT,78569
GO,61385
BA,50508
SP,45229
MA,42202
RS,40274
MS,33762
TO,33389


### Avaliar o Tempo de Resposta das Consultas

In [0]:
import time

# Exemplo: Medir tempo de execução para Consulta 1
start_time = time.time()

query1 = spark.sql("""
    SELECT uf, SUM(area_do_imovel) AS total_area_hectares
    FROM gold_temas_ambientais
    WHERE uf IN ('MS', 'MT')
    GROUP BY uf
    ORDER BY total_area_hectares DESC
""")
query1.show()

end_time = time.time()
print(f"Tempo de execução: {end_time - start_time:.2f} segundos")


+---+-------------------+
| uf|total_area_hectares|
+---+-------------------+
| MT|8.458708926979955E7|
| MS|3.632188266120003E7|
+---+-------------------+

Tempo de execução: 2.20 segundos
